In [ ]:
from importlib import reload
reload(change)
reload(base)

In [ ]:
import pandas as pd
import numpy as np
import src.in_out as io
import src.info as info
import src.auxiliary_functions as aux
import src.condition as con
import src.base as base
from itertools import combinations
from src.spread import get_spread
import src.change as change

## Veri Hazırlama

In [ ]:
#Verilere ait path'ler alınıyor
path_list=io.get_path('data')

In [ ]:
#Okunacak sütun adları
cols= ["Time","BID price","ASK price"]
#Tüm veriler okunuyor
master_data=io.read_allData(path_list, cols=cols)
master_data.columns=['time','bid_price','ask_price','name','date']

In [ ]:
#date sütunu Timestamp'e dönüştürülüyor ve tarih ve saat bilgisi birleştiriliyor
master_data['date']=pd.to_datetime(master_data['date'], errors='coerce')
master_data['date'] += pd.to_timedelta(master_data.pop('time').astype(str))

In [ ]:
#zaman dilimi ekleniyor
master_data['time_period']=master_data.date.dt.hour

In [ ]:
master_data

## Mid Price

In [ ]:
# mid price hesaplanıyor
master_data['mid_price']=base.average_of_series(master_data.bid_price,master_data.ask_price)

In [ ]:
# spread hesaplama için master_data, uygun bir forma dönüştürülüyor
df_mid_price=master_data.pivot(index='date', columns='name', values='mid_price')

## Spread
<img src="img/spread_flowchart.png">
<br/>
Spread, her bir saat için ayrı ayrı hesaplanmıştır

In [ ]:
values={'a_PNLTICK':1,
'a_TICKSIZE':1,
'b_PNLTICK':1,
'b_TICKSIZE':1}

In [ ]:
# pair'lar alınıyor
pairs=combinations(df_mid_price.columns, 2)

In [ ]:
#spread hesaplanıyor
spread_list=[
    base.groupby_date_time(df_mid_price.loc[:,pair]).apply(lambda x: get_spread(x,values))\
        for pair in pairs]
df_spread=pd.concat(spread_list,axis=1).droplevel([0,1])

## Change

In [ ]:
#change alınıyor
df_change=base.groupby_date_time(df_spread).apply(change.get_change)
df_change=df_change.droplevel([0,1])

## Amplitude

In [ ]:
all_pairs_of_amplitude=[]

In [ ]:
for pair_change in df_change:   #Her bir pair alınıyor
    pair_group = list(base.groupby_date_time(df_change[pair_change])) #gün ve saate göre gruplanıyor
    sub_amplitudes=[]
    for i in range(len(pair_group)):
        sub_change=base.to_series(pair_group[i][1])
        sub_amplitudes.append(change.get_amplitude(sub_change)) #pair'a ait her alt grubun amplitudu'u hesaplanıyor
    all_pairs_of_amplitude.append(pd.concat(sub_amplitudes)) # alt amplitude'lar birleştiriliyor

In [ ]:
from functools import reduce

In [ ]:
# Amplitude'u hesaplanan tüm pair'lar birleştiriliyor.
df_final = reduce(lambda left,right: pd.merge(left,right,on=['date','duration'],how='outer'), all_pairs_of_amplitude)

In [ ]:
df_final # NaN'lar birleştirmeden dolayı. ÖNEMSİZ ! 6AU8_6CU8, 2018-09-06 00:00:49 dan başladığından öncesi NaN

# Sonuçlar Excele Yazdırılıyor

In [ ]:
df_mid_price.to_excel('out/mid_price.xlsx')
df_spread.to_excel('out/spread.xlsx')
df_change.to_excel('out/change.xlsx')
df_final.to_excel('out/amplitude.xlsx')